In [5]:
import numpy as np
import cv2
import os
# import scipy
# import matplotlib.pylab as plt
# from skimage import io

In [6]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [7]:
def feature_object_detection(template, template_gray, frame, frame_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(frame_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.78*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])

    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 0.8) # H RANSAC
        # get the bounding box around template image
        h, w = template.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines(frame, [np.int32(transformed_box)], True, (0,0,255), 3, cv2.LINE_AA)
        drawmatch_img = cv2.drawMatchesKnn(template, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

            
        return detected_img, drawmatch_img
    else :
        print('Keypoints not enough')
        return

In [8]:
cap = cv2.VideoCapture('./Final_exam_6201072516311/videos/final_exam/Dataset-2/left_output.avi')
template = cv2.imread('./Final_exam_6201072516311/images/final_exam/Templates/Template-3.png')
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
while cap.isOpened() :
    #Read new frame
    ret, frame = cap.read()
    if ret == True :

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        detected, drawmatch =  feature_object_detection(template, template_gray, frame, frame_gray, 8)

        cv2.imshow('Frame', frame)
        
        if cv2.waitKey(int(1000/30)) & 0xFF == ord('q') : # Period control f - 1/T
            break
    else :
        break
    
cap.release()
cv2.destroyAllWindows()